# 3. Model Selection & Regularization

This notebook covers regularization techniques to prevent overfitting and improve predictions.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score, validation_curve

# Load data
data = pd.read_csv('./data/abide2.tsv', sep='\t')
features = data.filter(like='fs')
y = data['age']

## Bias-Variance Tradeoff

**Bias**: Systematic error from incorrect assumptions
- High bias → underfitting (too simple)
- Low bias → captures complex patterns

**Variance**: Sensitivity to training data fluctuations
- High variance → overfitting (too flexible)
- Low variance → stable predictions

**Key insight**: Reducing variance often requires accepting some bias (and vice versa).

## Regularization Principle

**Regularization** adds constraints to models based on prior knowledge:
- Introduces beneficial bias
- Reduces variance
- Prevents overfitting
- Improves generalization to new data

## Penalized Regression

### Ordinary Least Squares (OLS)
$$Cost = \sum_{i=1}^N (y_i - \sum_{j=1}^P \beta_j x_{ij})^2$$

### Lasso Regression (L1 penalty)
$$Cost = \sum_{i=1}^N (y_i - \sum_{j=1}^P \beta_j x_{ij})^2 + \lambda \sum_{j=1}^P |\beta_j|$$

- Shrinks coefficients toward zero
- Sets many coefficients exactly to zero → **feature selection**
- Best when few features are truly important

### Ridge Regression (L2 penalty)
$$Cost = \sum_{i=1}^N (y_i - \sum_{j=1}^P \beta_j x_{ij})^2 + \lambda \sum_{j=1}^P \beta_j^2$$

- Shrinks coefficients toward zero
- Never sets coefficients exactly to zero
- Best when many features contribute

**Parameter λ (alpha)**: Controls penalty strength
- λ = 0: no penalty (equivalent to OLS)
- Large λ: strong penalty (more regularization)

## Visualizing Coefficient Paths

In [ ]:
# Sample and standardize features
N_FEATURES = 200
X = features.sample(N_FEATURES, axis=1, random_state=99)
X_scaled = scale(X)

# Define helper function to plot coefficient paths
def plot_coef_path(estimator, X, y, alpha_range, title):
    coefs = np.zeros((X.shape[1], len(alpha_range)))
    for i, alpha in enumerate(alpha_range):
        model = estimator(alpha=alpha, max_iter=5000)
        model.fit(X, y)
        coefs[:, i] = model.coef_
    
    plt.figure(figsize=(10, 6))
    plt.plot(alpha_range, coefs.T, alpha=0.7)
    plt.xscale('log')
    plt.xlabel('Penalty (α)', fontsize=12)
    plt.ylabel('Coefficient Value', fontsize=12)
    plt.title(title, fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Lasso coefficient paths
alpha_lasso = np.logspace(-3, 1, 100)
plot_coef_path(Lasso, X_scaled, y, alpha_lasso, 
               'Lasso: Coefficients Shrink to Zero (Sparse Solution)')

In [ ]:
# Ridge coefficient paths
alpha_ridge = np.logspace(-5, 5, 100)
plot_coef_path(Ridge, X_scaled, y, alpha_ridge,
               'Ridge: Coefficients Shrink but Never Reach Zero (Dense Solution)')

## Tuning the Penalty Parameter

How do we choose the optimal penalty? Use **validation curves** to find the α that maximizes test performance.

In [ ]:
# Helper function to plot validation curves
def plot_validation_curve(model, X, y, param_range, model_name, baseline_r2=None):
    train_scores, test_scores = validation_curve(
        model, X, y,
        param_name='alpha',
        param_range=param_range,
        cv=5,
        scoring='r2'
    )
    
    train_mean = train_scores.mean(axis=1)
    test_mean = test_scores.mean(axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(param_range, train_mean, 'o-', label='Training', linewidth=2)
    plt.plot(param_range, test_mean, 'o-', label='Test (CV)', linewidth=2)
    
    if baseline_r2 is not None:
        plt.axhline(baseline_r2, linestyle='--', color='gray', 
                    linewidth=2, label='OLS (baseline)')
    
    plt.xscale('log')
    plt.xlabel('Penalty (α)', fontsize=12)
    plt.ylabel('R²', fontsize=12)
    plt.title(f'{model_name} Validation Curve', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1)
    
    # Find optimal alpha
    best_idx = test_mean.argmax()
    best_alpha = param_range[best_idx]
    best_score = test_mean[best_idx]
    
    plt.axvline(best_alpha, linestyle=':', color='red', alpha=0.7)
    plt.text(best_alpha, 0.05, f'Optimal α = {best_alpha:.4f}\nR² = {best_score:.3f}',
             fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.show()
    
    return best_alpha, best_score

In [ ]:
# Get OLS baseline for comparison
ols_r2 = cross_val_score(LinearRegression(), X_scaled, y, cv=5, scoring='r2').mean()
print(f"OLS baseline R²: {ols_r2:.3f}\n")

# Lasso validation curve
alpha_range_lasso = np.logspace(-3, 1, 30)
best_lasso_alpha, best_lasso_r2 = plot_validation_curve(
    Lasso(max_iter=5000), X_scaled, y, alpha_range_lasso, 'Lasso', ols_r2
)

print(f"\nLasso: Best α = {best_lasso_alpha:.4f}, R² = {best_lasso_r2:.3f}")
print(f"Improvement over OLS: {best_lasso_r2 - ols_r2:.3f}")

In [ ]:
# Ridge validation curve
alpha_range_ridge = np.logspace(0, 7, 50)
best_ridge_alpha, best_ridge_r2 = plot_validation_curve(
    Ridge(), X_scaled, y, alpha_range_ridge, 'Ridge', ols_r2
)

print(f"\nRidge: Best α = {best_ridge_alpha:.4f}, R² = {best_ridge_r2:.3f}")
print(f"Improvement over OLS: {best_ridge_r2 - ols_r2:.3f}")

## Comparing Models

Let's compare OLS, Lasso, and Ridge using their optimal parameters.

In [ ]:
models = [
    ('OLS', LinearRegression()),
    ('Lasso', Lasso(alpha=best_lasso_alpha, max_iter=5000)),
    ('Ridge', Ridge(alpha=best_ridge_alpha))
]

results = []
for name, model in models:
    scores = cross_val_score(model, X_scaled, y, cv=5, scoring='r2')
    results.append({
        'Model': name,
        'Mean R²': scores.mean(),
        'Std R²': scores.std()
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Visualize comparison
plt.figure(figsize=(8, 5))
plt.bar(results_df['Model'], results_df['Mean R²'], yerr=results_df['Std R²'],
        capsize=5, alpha=0.7, edgecolor='black')
plt.ylabel('Cross-Validated R²', fontsize=12)
plt.title('Model Comparison', fontsize=14)
plt.ylim(0, max(results_df['Mean R²']) * 1.2)
plt.grid(axis='y', alpha=0.3)
plt.show()

## Beyond Linear Models: Random Forests

**Random Forests** are ensembles of decision trees:
- Very flexible (can capture non-linear patterns)
- Handle high-dimensional data well
- Provide feature importances
- Can overfit if not properly tuned

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression

# Select 50 best features (reduces computational cost)
selector = SelectKBest(f_regression, k=50)
X_selected = selector.fit_transform(features, y)

# Train random forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_scores = cross_val_score(rf, X_selected, y, cv=5, scoring='r2')

print(f"Random Forest R²: {rf_scores.mean():.3f} ± {rf_scores.std():.3f}")

# Compare with Ridge on same features
ridge = Ridge(alpha=100)
ridge_scores = cross_val_score(ridge, scale(X_selected), y, cv=5, scoring='r2')

print(f"Ridge R²: {ridge_scores.mean():.3f} ± {ridge_scores.std():.3f}")

### Feature Importances

In [ ]:
# Fit random forest on 30 random features for interpretability
X_subset = features.sample(30, axis=1, random_state=42)
rf_interp = RandomForestRegressor(n_estimators=100, random_state=42)
rf_interp.fit(X_subset, y)

# Get feature importances
importances = pd.Series(rf_interp.feature_importances_, index=X_subset.columns)
top_features = importances.sort_values(ascending=False).head(10)

print("Top 10 Most Important Features:")
print(top_features)

# Plot
plt.figure(figsize=(10, 6))
top_features.plot(kind='barh', color='steelblue', edgecolor='black')
plt.xlabel('Feature Importance', fontsize=12)
plt.title('Random Forest: Top 10 Features', fontsize=14)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Summary

**Key Concepts:**

1. **Bias-Variance Tradeoff**: Accept some bias to reduce variance
2. **Regularization**: Add constraints to prevent overfitting
3. **Lasso**: Sparse solutions (feature selection)
4. **Ridge**: Dense solutions (all features contribute)
5. **Validation Curves**: Tune hyperparameters systematically

**Best Practices:**

- Always compare regularized models to OLS baseline
- Use cross-validation to tune penalty parameters
- Consider problem structure when choosing regularization:
  - Few important features → Lasso
  - Many contributing features → Ridge
- Try multiple model types (linear, tree-based, etc.)
- More data usually beats fancier algorithms

**Typical Workflow:**
1. Split data (train/test or use CV)
2. Try OLS baseline
3. Add regularization (Lasso/Ridge)
4. Tune hyperparameters via validation curves
5. Compare multiple model types
6. Select best model based on test performance
7. Interpret results cautiously (correlation ≠ causation)